## set up

In [168]:
# imports
import os
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
import json
from pydantic import BaseModel, Field
import string
import re
from typing import Optional 
import asyncio
import aiofiles
import time

In [169]:
# Define a semaphore to limit concurrency
SEMAPHORE_LIMIT = 20
semaphore = asyncio.Semaphore(SEMAPHORE_LIMIT)

async def semaphore_wrapper(func, *args):
    """Wrap async function with semaphore to limit concurrency"""
    async with semaphore:
        return await func(*args)

In [170]:
# api keys
try:
    with open("../secrets.json") as f:
        secrets = json.load(f)
    open_ai_key = secrets["openai"]
    anthropic_key = secrets["anthropic"]
    google_key = secrets["google"]
    os.environ["OPENAI_API_KEY"] = open_ai_key
    os.environ["ANTHROPIC_API_KEY"] = anthropic_key
    os.environ["GOOGLE_API_KEY"] = google_key
    print("API key loaded.")
except FileNotFoundError:
    print("Secrets file not found. YOU NEED THEM TO RUN THIS.")

API key loaded.


In [171]:
## make sure all folders are created inside notebooks folder
if not os.path.exists("scripts"):
    os.makedirs("scripts")
if not os.path.exists("json"):
    os.makedirs("json")
if not os.path.exists("chapters"):
    os.makedirs("chapters")

In [172]:
# NOT VERY RELEVANT
def format_gemini_response(res):
    output_formatted = res.content[7:-3]
    # turn into json
    output = json.loads(output_formatted)
    return output

class DotDict:
    # allow dot notation for dicts
    def __init__(self, dictionary):
        # check if dictionary is a string
        if isinstance(dictionary, str):
            # if it is a string, convert it to a dictionary
            print(dictionary)
            dictionary = json.loads(dictionary)
        for key, value in dictionary.items():
            setattr(self, key, value)

In [173]:
model_type = "default"
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)
# llm = ChatOpenAI(model="gpt-4o", temperature=0.1)
llm2 = ChatAnthropic(model_name="claude-3-5-sonnet-20240620", temperature=0.1, max_tokens_to_sample=4096, api_key=os.getenv("ANTHROPIC_API_KEY"))

# NOT USING RIGHT NOW
# model_type = "gemini"
llm3 = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.1)

In [174]:
# Pydantic
class StoryInfo(BaseModel):
    """Get overall information about the story."""
    numChapters: int = Field(description="Number of chapters in the story")
    firstline: str = Field(description="First line of the story")
    lastline: Optional[str] = Field(description="Last line of the story or first volume")
    markers: list[str] = Field(description="List of markers to split the story into chapters, as formatted in the table of contents. Make sure the formatting matches the markers in the actual text (including the word 'chapter' if applicable). Keep all punctuation marks and capitalization, and use the full chapter names. If there are multiple volumes, focus on the first volume.")

def get_marker_prompt(story):
    prompt = f'''List the markers needed to split this story into chapters: {story}. Format your response like Story Info:
    class StoryInfo(BaseModel):
        """Get overall information about the story."""
        firstline: str = Field(description="First line of the story")
        lastline: Optional[str] = Field(description="Last line of the story or first volume")
        markers: list[str] = Field(description="List of markers to split the story into chapters. Keep all punctuation marks and capitalization.")
    '''
    return prompt

# structured_llm = llm2.with_structured_output(StoryInfo)
structured_llm = llm.with_structured_output(StoryInfo)

## split text into chapters

In [175]:
# NOTE: Make sure your story file is in the scripts folder (as a txt file)
# read in txt file from scripts folder (inside notebooks folder)
og_story_name = "candide" # name of the story (should match the name of the file (e.g., type "color" for color.txt)
story_name = og_story_name
analysis_type = "theme" # character, theme

In [176]:
# optional when loading a new story, but this part needs to be manually looked up
# for image, provide a url or a file path to the image
story_info = {
    "title": "Candide",
    "type": "Novel",
    "author": "Voltaire",
    "year": "1759",
    "url": "https://www.gutenberg.org/ebooks/19942",
    "image": "https://m.media-amazon.com/images/I/61j3+Vjz0ZL._AC_UF1000,1000_QL80_.jpg"
}

In [177]:
# if json file already exists, use story info from it instead
if os.path.exists(f"../src/data/{og_story_name}-new.json"):
    with open(f"../src/data/{og_story_name}-new.json") as f:
        data = json.load(f)
    story_info = {}
    story_info["title"] = data["title"]
    story_info["type"] = data["type"]
    if (data["type"] == "Movie"):
        story_info["director"] = data["director"]
    else:
        story_info["author"] = data["author"]
    story_info["year"] = data["year"]
    if "url" in data:
        story_info["url"] = data["url"]
    if "image" in data:
        story_info["image"] = data["image"]

    print(story_info)

{'title': 'Candide', 'type': 'Novel', 'author': 'Voltaire', 'year': '1759', 'url': 'https://www.gutenberg.org/ebooks/19942', 'image': 'https://m.media-amazon.com/images/I/61j3+Vjz0ZL._AC_UF1000,1000_QL80_.jpg'}


In [178]:
# read in the script
with open(f"scripts/{story_name}.txt") as f:
    story = f.read()

In [179]:
if analysis_type == "theme":
    story_name += "_themes"
elif analysis_type == "2":
    story_name += "_2"

In [180]:
if not os.path.exists(f"json/{story_name}"):
    os.makedirs(f"json/{story_name}")

In [181]:
# focus on the first volume if there are multiple volumes
if "END OF VOL." in story:
    volumes = story.split("END OF VOL.")
    story = volumes[0]
elif "VOLUME II." in story:
    volumes = story.split("VOLUME II.")
    story = volumes[0]

In [182]:
# split story into chunks
char_chunk = 120000
story_len = len(story)
print(story_len)
if story_len > char_chunk:
    # story_chunks = [story[i:i + char_chunk] for i in range(0, len(story), char_chunk)]
    # take 0:char_chunk/2 and -char_chunk/2: for the first chunk
    chunk = story[0:char_chunk//2] + story[-char_chunk//2:]
    story_chunks = [chunk]
else: # if story is less than 120k characters, use the whole story
    story_chunks = [story]
print(len(story_chunks))

224844
1


In [183]:
all_res = []
if not os.path.exists(f"json/{og_story_name}/summary.json"):
    # only run if summary.json does not exist (no markers yet)
    for i, chunk in enumerate(story_chunks):
        invalid_input = True
        while invalid_input: # try until valid input
            # see if exception is thrown
            try:
                # if model_type == 'gemini':
                #     prompt = get_marker_prompt(story)
                #     markers = llm3.invoke(prompt)
                #     res = format_gemini_response(markers)
                #     print(res)
                #     res = DotDict(res['Story Info'])
                # else:
                res = structured_llm.invoke(chunk)
                all_res.append(res)
                print(f"Chunk {i}")
                invalid_input = False
            except Exception as e:
                print("Exception thrown. Trying again.")
                print("Error:", e)
                invalid_input = True
        # if i < len(story_chunks) - 1:
        #     # Pause for a specified time (e.g., 60 seconds) to avoid rate limits
        #     time.sleep(60)  # Adjust the sleep duration as per your rate limit requirements

In [184]:
if not os.path.exists(f"json/{og_story_name}/summary.json"):
    # save first line from first chunk, last line from last chunk, and combine markers
    first_line = all_res[0].firstline
    last_line = all_res[-1].lastline
    num_chapters = sum([res.numChapters for res in all_res])
    markers = []
    for res in all_res:
        markers.extend(res.markers)

    num_chapters, markers

In [185]:
if not os.path.exists(f"json/{og_story_name}/summary.json"):
    # checking output
    first_line, last_line

In [186]:
# Format the story
story = re.sub(r' {2,}', ' ', story)  # Replace consecutive spaces with one space
story = re.sub(r'\n{2,}', '\n', story)  # Replace consecutive newlines with one newline
story = re.sub(r' ?\n ?', '\n', story)  # Replace space + newline or newline + space with one newline
story = story.replace("“", "\"").replace("”", "\"").replace("‘", "'").replace("’", "'")

In [187]:
story_formatted = story.replace("\n", " ")
# replace weird quotes too
story_formatted = story_formatted.replace("\"", " ").replace("'", " ")

In [188]:
if not os.path.exists(f"json/{og_story_name}/summary.json"):
    # remove any marker that doesn't include one of these words (may or may not be necessary)
    exclude_words = ["part", "preface", "contents", "introduction", "end", "epilogue", "letter", "volume"]
    markers = [m for m in markers if not any(word in m.lower() for word in exclude_words)]

In [189]:
if not os.path.exists(f"json/{og_story_name}/summary.json"):
    # save as json if it doesn't exist yet
    story_json = {
        "first_line": first_line,
        "last_line": last_line,
        "markers": markers
    }

    if not os.path.exists(f"json/{og_story_name}"):
        os.makedirs(f"json/{og_story_name}")
    with open(f"json/{og_story_name}/summary.json", "w") as f:
        json.dump(story_json, f, indent=4)

In [190]:
# read in markers from summary.json
with open(f"json/{og_story_name}/summary.json") as f:
    summary = json.load(f)
    markers = summary["markers"]
    first_line = summary["first_line"]
    last_line = summary["last_line"]

In [191]:
story_formatted[-100:] 

'help produce our new eBooks, and how to subscribe to our email newsletter to hear about new eBooks. '

In [192]:
first_ind = story_formatted.find(first_line.replace("\"", " ").replace("'", " "))
first_ind, first_line.replace("\"", " ").replace("'", " ")

(10380,
 'In a castle of Westphalia, belonging to the Baron of Thunder-ten-Tronckh, lived a youth, whom nature had endowed with the most gentle manners.')

In [193]:
second_marker = markers[1] + " "
last_line_index = story_formatted.find(second_marker, first_ind)
last_line_index, second_marker, story_formatted[last_line_index-50:last_line_index+50]

(14864,
 'II WHAT BECAME OF CANDIDE AMONG THE BULGARIANS. ',
 'icent and most agreeable of all possible castles. II WHAT BECAME OF CANDIDE AMONG THE BULGARIANS. Ca')

In [194]:
last_ind = story_formatted.find(last_line.replace("\"", " ").replace("'", " "))
last_ind, last_line.replace("\"", " ").replace("'", " ")

(192650,
 ' All that is very well,  answered Candide,  but let us cultivate our garden. ')

In [195]:
# crop story to first_ind and last_ind
story = story[first_ind:last_ind + len(last_line)]

In [196]:
# crop story_formatted too
story_formatted = story_formatted[first_ind:last_ind + len(last_line)]

In [197]:
story[-100:], story_formatted[-100:]

('s and pistachio-nuts."\n"All that is very well," answered Candide, "but let us cultivate our\ngarden."',
 's and pistachio-nuts.   All that is very well,  answered Candide,  but let us cultivate our garden. ')

In [198]:
# Only run if chapters folder does not exist yet
if not os.path.exists(f"chapters/{og_story_name}"):
    # split story into chapters based on markers
    chapters = []
    cur_first_line = first_line.replace("\"", " ").replace("'", " ")
    cur_min_index = 0
    for i, marker in enumerate(markers):
        # print("marker", marker)
        exists_next = i+1 < len(markers)
        next_marker = markers[i+1] if exists_next else ""
        # print("next_marker", next_marker)
        
        if exists_next and " ACT " in next_marker.upper():
            next_marker = next_marker.upper()
        formatted_last_line = next_marker if exists_next else last_line
        # print("formatted_last_line", formatted_last_line)

        if " ACT " not in next_marker.upper():
            formatted_last_line += " "
        cur_last_line = formatted_last_line
        cur_last_line = cur_last_line.replace("\"", " ").replace("'", " ")
        first_line_index = story_formatted.find(cur_first_line, cur_min_index)
        print("cur_first_line", repr(cur_first_line))
        print("first_line_index", first_line_index)
        print("Snippet around first_line_index:", repr(story_formatted[first_line_index:first_line_index + 200]))

        if i != 0 and " ACT " not in marker.upper():
            first_line_index += len(cur_first_line)
        last_line_index = story_formatted.find(cur_last_line, first_line_index)
        print("cur_last_line", repr(cur_last_line))
        print("last_line_index", last_line_index)
        print("Snippet around last_line_index:", repr(story_formatted[last_line_index-200:last_line_index]))
        
        if i+1 == len(markers):
            if last_line_index == -1:
                last_line_index = len(story_formatted)
            else:
                last_line_index += len(cur_last_line) 
        chapter = story[first_line_index:last_line_index].strip()

        # add line number in front of each line
        chapter = chapter.split("\n")
        for j, line in enumerate(chapter):
            chapter[j] = f"LINE {j+1}: {line}"
        chapter = "\n".join(chapter)
        chapters.append(chapter)
        cur_first_line = cur_last_line
        cur_min_index = last_line_index

In [199]:
if not os.path.exists(f"chapters/{og_story_name}"):
    # check results
    for i, marker in enumerate(markers):
        print(f"Chapter {marker}")
        chapter_first_line = chapters[i][:100]
        chapter_last_line = chapters[i][-100:]
        print("first_line:", chapter_first_line)
        print("last_line:", chapter_last_line)
        print("\n")

In [200]:
if not os.path.exists(f"chapters/{og_story_name}"):
    # create folder for story if it doesn't exist
    os.makedirs(f"chapters/{og_story_name}", exist_ok=True)

    # also make chapter folder in public directory
    os.makedirs(f"../public/chapters/{og_story_name}", exist_ok=True)

    # save chapters to new txt files
    for i, marker in enumerate(markers):
        with open(f"chapters/{og_story_name}/{marker}.txt", "w") as f:
            f.write(chapters[i])
            print(f"Chapter {marker} saved.")
        # save to public folder too
        with open(f"../public/chapters/{og_story_name}/{marker}.txt", "w") as f:
            f.write(chapters[i])
            print(f"Chapter {marker} saved to public folder.")

In [201]:
# read in chapters from txt files
chapters = []
for i, marker in enumerate(markers):
    with open(f"chapters/{og_story_name}/{marker}.txt") as f:
        chapter = f.read()
        chapters.append(chapter)

In [202]:
chapters[0]

'LINE 1: In a castle of Westphalia, belonging to the Baron of\nLINE 2: Thunder-ten-Tronckh, lived a youth, whom nature had endowed with the\nLINE 3: most gentle manners. His countenance was a true picture of his soul. He\nLINE 4: combined a true judgment with simplicity of spirit, which was the\nLINE 5: reason, I apprehend, of his being called Candide. The old servants of\nLINE 6: the family suspected him to have been the son of the Baron\'s sister, by\nLINE 7: a good, honest gentleman of the neighborhood, whom that young lady would\nLINE 8: never marry because he had been able to prove only seventy-one\nLINE 9: quarterings, the rest of his genealogical tree having been lost through\nLINE 10: the injuries of time.\nLINE 11: The Baron was one of the most powerful lords in Westphalia, for his\nLINE 12: castle had not only a gate, but windows. His great hall, even, was hung\nLINE 13: with tapestry. All the dogs of his farm-yards formed a pack of hounds at\nLINE 14: need; his grooms were h

## analyze scene

In [203]:
# check if all.json exists, this will determine if we need to generate all scene details or just character details
scene_details_exist = os.path.exists(f"json/{og_story_name}/all.json")
scene_details_exist

True

In [204]:
scene_type = "location" # location, character, or n/a

In [205]:
extra_scene_instruction = "Break this chapter into scenes, starting at line 1."
if scene_type == "location":
    extra_scene_instruction += " Do not start a new scene until the location changes. Record the line number when this happens."
elif scene_type == "character":
    extra_scene_instruction += " Do not start a new scene until a new character enters or an old character leaves. Record the line number when this happenes."

extra_scene_instruction += "\nYour goal is to have as few scenes as possible.\nChapter text:"

extra_scene_instruction

'Break this chapter into scenes, starting at line 1. Do not start a new scene until the location changes. Record the line number when this happens.\nYour goal is to have as few scenes as possible.\nChapter text:'

In [206]:
# Pydantic
class SceneInfo(BaseModel):
    """Get information about a scene in the story."""
    scene: str = Field(description="Short title of scene (don't include chapter name or number). Each scene should have a unique title, and each title should be no longer than 6 words.")
    summary: str = Field(description="1 line summary of the scene")
    firstline: int = Field(description="First line number of the current scene in this chapter (should be after last line number of previous scene)")
    lastline: int = Field(description="Last line number of the current scene in this chapter (should be after first line number of current scene)")
    explanation: str = Field(description="Explanation of why a new scene starts here. For the first scene in the chapter, just write 'Start of chapter'.")

class SceneListInfo(BaseModel):
    """Get overall information about the story."""
    scenelist: list[SceneInfo] = Field(description="List of key scenes in this chapter")

def get_scenelist_prompt(chapter):
    scene_prompt = f'''List the key scenes in this chapter: {chapter}. Format your response like SceneListInfo:
                    class SceneInfo(BaseModel):
                        """Get information about a scene in the story."""
                        scene: str = Field(description="Title of scene (don't include chapter name or number)")
                        summary: str = Field(description="1 line summary of the scene")
                        firstline: int = Field(description="First line number of the current scene in this chapter (should be after last line number of previous scene)")
                        lastline: int = Field(description="Last line number of the current scene in this chapter (should be after first line number of current scene)")

                    class SceneListInfo(BaseModel):
                        """Get overall information about the story."""
                        scenelist: list[SceneInfo] = Field(description="List of key scenes in this chapter")'''
    return scene_prompt

scene_llm = llm.with_structured_output(SceneListInfo)
# scene_llm = llm2.with_structured_output(SceneListInfo)

In [207]:
# Pydantic
class CharacterInfo(BaseModel):
    """Get information about a character in this scene."""
    name: str = Field(description="Full name of the character. Character name should be 5 words or less.")
    # role: str = Field(description="Main actions and/or motivations of the character in this scene")
    importance: float = Field(description="Importance of the character in this scene from from 0: not important at all to 1: very important character")
    importance_exp: str = Field(description="Brief, 1-phrase explanation your importance rating for this character")
    emotion: str = Field(description="Emotion of the character in this scene, described in a few words or a short phrase")
    sentiment: float = Field(description="Sentiment of the character in this scene from -1 (very negative) to 1 (very positive)")
    quote: str = Field(description="A direct quote from the character in this scene to illustrate their emotions and/or actions")
    fake_quote: str = Field(description="A brief explanation of the character's emotions and/or actions in this scene if you cannot find a direct quote in the text. If you can find a direct quote, leave this field empty.")

# Pydantic
class ThemeInfo(BaseModel):
    """Get information about a theme in this scene."""
    name: str = Field(description="Short general phrase to describe the theme. Theme name should be 3 words or less.")
    # role: str = Field(description="Short description of the role of the theme in this scene")
    importance: float = Field(description="Importance of the theme in this scene from from 0: not important at all to 1: very important theme")
    importance_exp: str = Field(description="Brief, 1-phrase explanation of your importance rating for this theme")
    emotion: str = Field(description="Emotion associated with the theme in this scene, described in a few words or a short phrase")
    sentiment: float = Field(description="Sentiment of the theme in this scene from -1 (very negative) to 1 (very positive)")
    quote: str = Field(description="A direct quote in this scene to illustrate the theme")
    fake_quote: str = Field(description="A brief explanation of the theme in this scene if you cannot find a direct quote in the text. If you can find a direct quote, leave this field empty.")

class SceneDetails(BaseModel):
    """Get overall information about the scene."""
    location: str = Field(description="Physical location of the scene. Location name should be 5 words or less.")
    importance: float = Field(description="Importance of the scene from 0: not important at all to 1: very important and pivotal scene")
    conflict: float = Field(description="Conflict in the scene from 0: no conflict to 1: high conflict")
    characters: list[CharacterInfo] = Field(description="List of key characters in the scene. Make sure there are no duplicates, and focus on speaking characters. Don't include any unnamed characters.")

class SceneDetailsBare(BaseModel):
    """Get overall information about the scene."""
    characters: list[CharacterInfo] = Field(description="List of key characters in the scene. Make sure there are no duplicates, and focus on speaking characters. Don't include any unnamed characters.")

class SceneDetailsTheme(BaseModel):
    """Get overall information about the scene."""
    location: str = Field(description="Physical location of the scene. Location name should be 5 words or less.")
    importance: float = Field(description="Importance of the scene from 0: not important at all to 1: very important and pivotal scene")
    conflict: float = Field(description="Conflict in the scene from 0: no conflict to 1: high conflict")
    themes: list[ThemeInfo] = Field(description="List of key themes in the scene. Make sure there are no duplicates but try to make the themes as general as possible.")

class SceneDetailsBareTheme(BaseModel):
    """Get overall information about the scene."""
    themes: list[ThemeInfo] = Field(description="List of key themes in the scene. Make sure there are no duplicates but try to make the themes as general as possible.")

def get_scenedetails_prompt(scene):
    scene_prompt = f'''List the key details in this scene: {scene}. Format your response like SceneDetails:
                    class CharacterInfo(BaseModel):
                        """Get information about a character in this scene."""
                        name: str = Field(description="Full name of the character. Character name should be 5 words or less.")
                        role: str = Field(description="Main actions and/or motivations of the character in this scene")
                        importance: float = Field(description="Importance of the character in this scene from from 0: not important at all to 1: very important character")
                        emotion: str = Field(description="Emotion of the character in this scene, described in a few words or a short phrase")
                        sentiment: float = Field(description="Sentiment of the character in this scene from -1 (very negative) to 1 (very positive)")
                        quote: str = Field(description="A direct quote from the character in this scene to illustrate their emotions and/or actions")

                    class SceneDetails(BaseModel):
                        """Get overall information about the scene."""
                        location: str = Field(description="Physical location of the scene. Location name should be 5 words or less.")
                        importance: float = Field(description="Importance of the scene from 0: not important at all to 1: very important and pivotal scene")
                        conflict: float = Field(description="Conflict in the scene from 0: no conflict to 1: high conflict")
                        characters: list[CharacterInfo] = Field(description="List of key characters in the scene. Make sure there are no duplicates, and focus on speaking characters.  Don't include any unnamed characters.")'''
    return scene_prompt

scene_details_llm = llm.with_structured_output(SceneDetails)
scene_details_llm_bare = llm.with_structured_output(SceneDetailsBare)
scene_details_llm_theme = llm.with_structured_output(SceneDetailsTheme)
scene_details_llm_bare_theme = llm.with_structured_output(SceneDetailsBareTheme)
# scene_details_llm = llm2.with_structured_output(SceneDetails)

In [208]:
async def get_scene_info_async(chapters, chapter_num):
    print(f"Splitting Chapter {markers[chapter_num]} into scenes...")

    cur_chapter = chapters[chapter_num]

    invalid_input = True
    while invalid_input: # try until valid input
        # see if exception is thrown
        try:
            if model_type == "gemini":
                prompt = get_scenelist_prompt(cur_chapter)
                chapter_scenes = await llm3.ainvoke(prompt)
                chapter_scenes = format_gemini_response(chapter_scenes)
            else:
                chapter_scenes = await scene_llm.ainvoke(extra_scene_instruction + "\n" + cur_chapter)
            invalid_input = False
        except Exception as e:
            print("Exception thrown. Trying again.")
            print("Error:", e)
            invalid_input = True
            
    if model_type == "gemini":
        chapter_scenes_list = [DotDict(scene) for scene in chapter_scenes['scenelist']]
    else: 
        chapter_scenes_list = chapter_scenes.scenelist
    formatted_chapter_scenes = []

    cur_last_line = 0

    for scene in chapter_scenes_list:
        scene_title = scene.scene
        scene_summary = scene.summary
        scene_first_line = scene.firstline
        scene_last_line = scene.lastline
        scene_exp = scene.explanation

        print(f"FIRST LINE: {scene_first_line}")
        print(f"LAST LINE: {scene_last_line}")
        print()

        formatted_scene = {
            "title": scene_title,
            "summary": scene_summary,
            "chapter": markers[chapter_num],
            "first_line": scene_first_line,
            "last_line": scene_last_line,
            "explanation": scene_exp
        }

        if scene_last_line > scene_first_line and scene_first_line > cur_last_line:
            # only add scene if last_line > first_line and first_line > cur_last_line
            formatted_chapter_scenes.append(formatted_scene)
            formatted_chapter = cur_chapter.replace("\n", " ")
            # replace weird quotes too
            formatted_chapter = formatted_chapter.replace("\"", " ").replace("'", " ")
            cur_last_line = scene_last_line

    # sort scenes by first_line
    formatted_chapter_scenes = sorted(formatted_chapter_scenes, key=lambda x: x["first_line"])

    # split chapter into scenes based on formatted_chapter_scenes
    scenes = []
    cur_min_index = 0
    for i, scene in enumerate(formatted_chapter_scenes):
        cur_first_line = f"LINE {scene['first_line']}:"
        # cur_first_line = formatted_chapter_scenes[i]["first_line"].strip("...").replace("\"", " ").replace("'", " ")
        first_line_index = formatted_chapter.find(cur_first_line, cur_min_index)
        if first_line_index == -1:
            # see if first_line_index overlaps with last_line of previous scene
            other_possible_first_index = formatted_chapter.find(cur_first_line)
            if other_possible_first_index != -1:
                overlap = cur_chapter[other_possible_first_index:cur_min_index]
                print(f"SCENE {i}")
                print("\nPREVIOUS SCENE")
                print(scenes[i-1])
                print("\nOVERLAP")
                print(overlap)
                print()
            first_line_index = cur_min_index
        cur_last_line = f"LINE {scene['last_line'] + 1}:"
        # cur_last_line = formatted_chapter_scenes[i]["last_line"].strip("...").replace("\"", " ").replace("'", " ")
        last_line_index = formatted_chapter.find(cur_last_line, first_line_index)
        # last_line_index += len(cur_last_line)
        if last_line_index == -1:
            last_line_index = len(cur_chapter)
        scene_text = cur_chapter[first_line_index:last_line_index].strip()
        # remove line numbers
        scene_text = re.sub(r'LINE \d+: ', '', scene_text)
        scenes.append(scene_text)
        cur_min_index = last_line_index

    # add scene text to formatted_chapter_scenes
    for i, scene in enumerate(formatted_chapter_scenes):
        formatted_chapter_scenes[i]["text"] = scenes[i]

    return scenes, formatted_chapter_scenes

In [209]:
async def get_scene_details_async(scenes, formatted_chapter_scenes, chapter_num):
    print(f"Generating scene details for Chapter {markers[chapter_num]}...")
    all_scene_details = []

    async def process_scene(scene_text, cur_scene):
        """Async function to analyze a single scene"""
        if len(scene_text) == 0:
            return
        
        start_prompt = """
                          When analyzing this scene, focus on characters with a significant role in the scene.
                          If you can't determine the character's role or emotion, don't include them in the analysis.

                          Make sure to only use direct, continuous quotes from the text.
                          Don't paraphrase or change the text in any way (e.g., don't join multiple quotes together).
                          If the character speaks, the quote should be something they say.
                          If the character is only described, the quote should be a description of the character's actions or emotions
                          (or any quote that mentions them, even if it is said by another character).
                          Try your best to find a continous quote for each character, but if you can't find a quote, 
                          (1) set the "quote" field to "No quote available" and (2) in the "fake_quote" field,
                          write a brief explanation of the character's emotions and actions in the scene.
                          Leave the "fake_quote" field empty if you can find a continous quote.
                        
                          Scene text:

                          """
        if analysis_type == "theme":
            start_prompt = """
                            When analyzing this scene, focus on themes with a significant role in the scene.
                            If you can't determine the theme's role or emotion, don't include it in the analysis.

                            Make sure to only use direct, continuous quotes from the text that describe the theme or relate to it.
                            Don't paraphrase or change the text in any way (e.g., don't join multiple quotes together).
                            Try your best to find a continuous quote for each theme, but if you can't find a quote,
                            (1) set the "quote" field to "No quote available" and (2) in the "fake_quote" field,
                            write a brief explanation of the theme in this scene.
                            Leave the "fake_quote" field empty if you can find a continuous quote.
                        
                            Scene text:

                            """
            
        invalid_input = True
        while invalid_input:
            try:
                if analysis_type == "theme":
                    scene_details = await scene_details_llm_theme.ainvoke(start_prompt + cur_scene)
                else:
                    scene_details = await scene_details_llm.ainvoke(start_prompt + cur_scene)
                invalid_input = False
                # print(scene_details)
            except Exception as e:
                print("Exception thrown. Trying again.")
                print("Error:", e)
                invalid_input = True
        
        return scene_details

    # Run all scene details in parallel
    tasks = [process_scene(formatted_chapter_scenes[i]["text"], scenes[i]) for i in range(len(scenes))]
    all_scene_details = await asyncio.gather(*tasks)

    # add details to formatted_chapter_scenes
    for i, scene in enumerate(formatted_chapter_scenes):
        scene_text = formatted_chapter_scenes[i]["text"]
        text_length = len(scene_text)
        num_lines = formatted_chapter_scenes[i]["last_line"] - formatted_chapter_scenes[i]["first_line"] + 1
        # first_line = formatted_chapter_scenes[i]["first_line"]
        # first_line_length = len(first_line)
        # last_line = formatted_chapter_scenes[i]["last_line"]
        # last_line_length = len(last_line)

        # if text_length == 0 or text_length < first_line_length + last_line_length:
        if text_length == 0:
            # no scene text or scene text is too short
            continue
        
        cur_scene_details = all_scene_details[i]
        formatted_chapter_scenes[i]["length"] = text_length
        formatted_chapter_scenes[i]["num_lines"] = num_lines
        formatted_chapter_scenes[i]["location"] = cur_scene_details.location
        formatted_chapter_scenes[i]["importance"] = cur_scene_details.importance
        formatted_chapter_scenes[i]["conflict"] = cur_scene_details.conflict
        formatted_character_info = []

        characters = cur_scene_details.themes if analysis_type == "theme" else cur_scene_details.characters
        # if model_type == "gemini":
        #     characters = [DotDict(character) for character in characters]
        for character in characters:
            character_info = {
                "name": character.name,
                "importance": character.importance,
                "importance_exp": character.importance_exp,
                "emotion": character.emotion,
                "sentiment": character.sentiment,
                "quote": character.quote,
                "fake_quote": character.fake_quote
            }
            # add character only if emotion is not N/A and quote is not "No quote available"
            if not (character.emotion == "N/A" and "No quote available" in character.quote):
                formatted_character_info.append(character_info)
        formatted_chapter_scenes[i]["characters"] = formatted_character_info

    return formatted_chapter_scenes

In [210]:
async def replace_characters_only_async():
    # read in data from all.json
    with open(f"json/{og_story_name}/all.json") as f:
        scenes = json.load(f)
    
    async def process_scene(scene_info, scene_text):
        """Async function to replace scene details with characters only"""

        if len(scene_text) == 0:
            return
        
        start_prompt = """
                          When analyzing this scene, focus on characters with a significant role in the scene.
                          If you can't determine the character's role or emotion, don't include them in the analysis.

                          Make sure to only use direct, continuous quotes from the text.
                          Don't paraphrase or change the text in any way (e.g., don't join multiple quotes together).
                          If the character speaks, the quote should be something they say.
                          If the character is only described, the quote should be a description of the character's actions or emotions
                          (or any quote that mentions them, even if it is said by another character).
                          Try your best to find a continous quote for each character, but if you can't find a quote, 
                          (1) set the "quote" field to "No quote available" and (2) in the "fake_quote" field,
                          write a brief explanation of the character's emotions and actions in the scene.
                          Leave the "fake_quote" field empty if you can find a continous quote.
                        
                          Scene text:

                          """
        if analysis_type == "theme":
            start_prompt = """
                            When analyzing this scene, focus on themes with a significant role in the scene.
                            If you can't determine the theme's role or emotion, don't include it in the analysis.

                            Make sure to only use direct, continuous quotes from the text that describe the theme or relate to it.
                            Don't paraphrase or change the text in any way (e.g., don't join multiple quotes together).
                            Try your best to find a continuous quote for each theme, but if you can't find a quote,
                            (1) set the "quote" field to "No quote available" and (2) in the "fake_quote" field,
                            write a brief explanation of the theme in this scene.
                            Leave the "fake_quote" field empty if you can find a continuous quote.
                        
                            Scene text:

                            """
            
        invalid_input = True
        while invalid_input:
            try:
                if analysis_type == "theme":
                    scene_details = await scene_details_llm_bare_theme.ainvoke(start_prompt + scene_text)
                else:
                    scene_details = await scene_details_llm_bare.ainvoke(start_prompt + scene_text)
                invalid_input = False
                # print(scene_details)
            except Exception as e:
                print("Exception thrown. Trying again.")
                print("Error:", e)
                invalid_input = True
            
        characters = scene_details.themes if analysis_type == "theme" else scene_details.characters
        # if model_type == "gemini":
        #     characters = [DotDict(character) for character in characters]
        new_characters = []
        for character in characters:
            character_info = {
                "name": character.name,
                "importance": character.importance,
                "importance_exp": character.importance_exp,
                "emotion": character.emotion,
                "sentiment": character.sentiment,
                "quote": character.quote,
                "fake_quote": character.fake_quote
            }
            # add character only if emotion is not N/A and quote is not "No quote available"
            if not (character.emotion == "N/A" and "No quote available" in character.quote):
                new_characters.append(character_info)
        scene_info["characters"] = new_characters

        print(f"Processed Chapter {scene_info['chapter']}, Scene {scene_info['title']}.")
        
        return scene_info

    # Run all scene details in parallel
    tasks = [semaphore_wrapper(process_scene, scenes[i], scenes[i]["text"]) for i in range(len(scenes))]
    all_scene_details = await asyncio.gather(*tasks)

    # save to all.json
    with open(f"json/{story_name}/all.json", "w") as f:
        json.dump(all_scene_details, f, indent=4)
    print(f"Scene data updated with new {analysis_type} data. Saved to all.json.")

In [211]:
async def process_all_chapters_async():
    if scene_details_exist:
        print(f"Scene details already exist. Only replacing {analysis_type} info.")
        await replace_characters_only_async()
    else:
        # Step 1: Split each chapter into scenes in parallel
        chapter_tasks = [get_scene_info_async(chapters, i) for i in range(len(chapters))]
        chapter_results = await asyncio.gather(*chapter_tasks)

        # extract scenes and formatted_chapter_scenes
        scenes = [result[0] for result in chapter_results]
        formatted_chapter_scenes = [result[1] for result in chapter_results]

        # Step 2: Get scene details for each scene in parallel
        detail_tasks = [semaphore_wrapper(get_scene_details_async, scenes[i], formatted_chapter_scenes[i], i) for i in range(len(chapters))]
        final_formatted_scenes = await asyncio.gather(*detail_tasks)

        # Step 3: Filter and save results
        for i, marker in enumerate(markers):
            formatted_chapter_scenes = final_formatted_scenes[i]

            # remove scenes with no text
            formatted_chapter_scenes = [scene for scene in formatted_chapter_scenes if scene["text"] != "" and "characters" in scene]
            # remove characters with no quote
            for scene in formatted_chapter_scenes:
                scene["characters"] = [character for character in scene["characters"] if "(" not in character["quote"] != "" and "<" not in character["name"] and "unnamed" not in character["name"].lower()]
                # print(scene)
            
            # remove scenes with no characters
            formatted_chapter_scenes = [scene for scene in formatted_chapter_scenes if len(scene["characters"]) > 0]

            # save as json
            with open(f"json/{story_name}/{marker}.json", "w") as f:
                json.dump(formatted_chapter_scenes, f, indent=4)

            print(f"All details for Chapter {marker} saved.\n")

In [212]:
start_time = time.time()

In [213]:
# RUN MAIN SCENE LOOP ASYNC that will get all the scene/character/etc. info for each chapter
await process_all_chapters_async()

Scene details already exist. Only replacing theme info.
Processed Chapter III HOW CANDIDE MADE HIS ESCAPE FROM THE BULGARIANS, AND WHAT AFTERWARDS BECAME OF HIM., Scene Destruction of Villages.
Processed Chapter II WHAT BECAME OF CANDIDE AMONG THE BULGARIANS., Scene Pardon from the King.
Processed Chapter III HOW CANDIDE MADE HIS ESCAPE FROM THE BULGARIANS, AND WHAT AFTERWARDS BECAME OF HIM., Scene Encounter with a Beggar.
Processed Chapter III HOW CANDIDE MADE HIS ESCAPE FROM THE BULGARIANS, AND WHAT AFTERWARDS BECAME OF HIM., Scene Arrival in Holland.
Processed Chapter V TEMPEST, SHIPWRECK, EARTHQUAKE, AND WHAT BECAME OF DOCTOR PANGLOSS, CANDIDE, AND JAMES THE ANABAPTIST., Scene Philosophical Debate.
Processed Chapter II WHAT BECAME OF CANDIDE AMONG THE BULGARIANS., Scene Forced into the Regiment.
Processed Chapter III HOW CANDIDE MADE HIS ESCAPE FROM THE BULGARIANS, AND WHAT AFTERWARDS BECAME OF HIM., Scene Battlefield Carnage.
Processed Chapter I HOW CANDIDE WAS BROUGHT UP IN A MAG

In [214]:
# rank each scene by importance
# and within each scene, the characters
if not scene_details_exist:
    for i, marker in enumerate(markers):
        print(marker)
        with open(f"json/{story_name}/{marker}.json") as f:
            scenes = json.load(f)
        importances = []
        conflicts = []
        
        # extract importance from each scene
        for j, scene in enumerate(scenes):
            importances.append((j, scene["importance"]))
            conflicts.append((j, scene["conflict"]))
            # add number of each scene
            scenes[j]["number"] = j+1
        # sort importances
        sorted_importances = sorted(importances, key=lambda x: x[1], reverse=True)
        # add importance_rank to each scene
        for k, (j, _) in enumerate(sorted_importances):
            scenes[j]["importance_rank"] = k+1
        # sort conflicts
        sorted_conflicts = sorted(conflicts, key=lambda x: x[1], reverse=True)
        # add conflict_rank to each scene
        for k, (j, _) in enumerate(sorted_conflicts):
            scenes[j]["conflict_rank"] = k+1
        
        # print results
        # for j, scene in enumerate(scenes):
        #     print(scenes[j]["title"], scenes[j]["importance_rank"])
        #     print("--------------------------------")
        #     for k, character in enumerate(scene["characters"]):
        #         print(character["name"], character["importance_rank"])
        #     print()

        # save as json
        with open(f"json/{story_name}/{marker}.json", "w") as f:
            json.dump(scenes, f, indent=4)

In [215]:
# combine all chapters into one json file
if not scene_details_exist:
    all_scenes = []
    for i, marker in enumerate(markers):
        with open(f"json/{story_name}/{marker}.json") as f:
            scenes = json.load(f)
        all_scenes += scenes

    with open(f"json/{story_name}/all.json", "w") as f:
        json.dump(all_scenes, f, indent=4)

### grouping duplicate characters / location names

In [216]:
# collect all unique characters and locations in the story
all_characters = {}
all_locations = []

if not scene_details_exist:
    for i, marker in enumerate(markers):
        with open(f"json/{story_name}/{marker}.json") as f:
            scenes = json.load(f)
        for scene in scenes:
            for character in scene["characters"]:
                char = character["name"]
                if char not in all_characters:
                    all_characters[char] = character["importance_exp"]
            location = scene["location"]
            if location not in all_locations:
                all_locations.append(location)
else:
    # load from all.json
    with open(f"json/{story_name}/all.json") as f:
        scenes = json.load(f)
    for scene in scenes:
        for character in scene["characters"]:
            char = character["name"]
            if char not in all_characters:
                all_characters[char] = character["importance_exp"]
        location = scene["location"]
        if location not in all_locations:
            all_locations.append(location)

In [217]:
len(all_characters), len(all_locations)

(304, 90)

In [218]:
all_characters

{'Optimism': 'This theme is central to the philosophy presented in the scene.',
 'Social Status': "The characters' social standings are crucial to the setting and interactions.",
 'Innocence': "Candide's innocence shapes his perceptions and beliefs.",
 'Curiosity for Knowledge': "Cunegonde's desire to learn is central to her character development.",
 'Romantic Aspirations': "Cunegonde's thoughts about Candide reflect her romantic feelings.",
 'Forbidden Love': 'This theme drives the conflict in the scene.',
 'Social Rejection': 'The theme highlights the consequences of their actions.',
 'Family Conflict': 'The theme illustrates the familial repercussions of their romance.',
 'Despair': "This theme is central to Candide's experience in the scene.",
 'Isolation': "Candide's solitude emphasizes his emotional state.",
 'Suffering': 'The physical and emotional suffering is a key aspect of the scene.',
 'Generosity': 'This theme is central to the interactions in the scene.',
 'Community Supp

In [219]:
all_locations

["Baron Of Thunder-ten-tronckh's Castle",
 'Castle Park',
 'Behind The Screen',
 'Field Near Waldberghofftrarbk-dikdorff',
 'Dining Table',
 'Regiment Barracks',
 'Training Grounds',
 'Dungeon',
 'Kingdom Of The Bulgarians',
 'Battlefield',
 'Holland',
 'Public Square',
 "James' Home",
 'City Street',
 "Anabaptist's Stable",
 'Ship To Lisbon',
 'Inquisition Chamber',
 'Lisbon',
 'Unknown Location',
 'Decayed House',
 'Lonely House In The Country',
 "Cunegonde's House",
 "Cunegonde's Story",
 'On The Sofa',
 "Cunegonde's Feet",
 'Stable',
 'Public Inn In Avacena',
 'On The Voyage',
 "Old Woman's Home",
 'Massa Carara',
 'On Board The Pirate Vessel',
 'Morocco',
 'Orange Tree By Rivulet',
 'Algiers',
 'Fort On Palus Méotides',
 'On Board A Ship',
 'Buenos Ayres',
 'On The Road To Paraguay',
 "Commandant's Arbour",
 'Chapel Of The Jesuits',
 'Beautiful Meadow',
 'Wood Near Buenos Ayres',
 'Frontiers Of The Oreillons',
 'Near A Little River',
 'First Village',
 "Landlord's Residence",
 'Pl

In [220]:
# # create csv file for characters
# import csv

# with open(f"csv/{story_name}_characters.csv", "w", newline="") as f:
#     writer = csv.writer(f)
#     writer.writerow(["Character", "Role"])
#     for char, role in all_characters.items():
#         writer.writerow([char, role])

In [221]:
# # create csv file for locations
# with open(f"csv/{story_name}_locations.csv", "w", newline="") as f:
#     writer = csv.writer(f)
#     writer.writerow(["Location"])
#     for location in all_locations:
#         writer.writerow([location])

In [222]:
# Assuming your JSON file is stored at 'all.json'
with open(f"json/{story_name}/all.json") as f:
    json_data = json.load(f)

prompt = """
You are given a list of characters with some duplicates or alternate names.
Your task is to group all the alternate names for each unique character under a single full name, using the descriptions as guidance.
Don't keep characters like "Narrator" unless they are never referred to by another name.
Separate characters that are related to others (e.g., "Bob" and "Ms. Doe (Bob's mom)" and "Bob's sister") into separate entries.

First create a new character list by removing all duplicates and alternate names and only keeping the full name for each character.

Then, output a JSON dictionary where the key is each character in your new list and the value is a list of all possible alternate names. 
All names should be taken directly from the original character list and should only appear once in the output.

For example:
{{
    "Bob Smith": ["Bob", "Bobby", "Robert"],
    "Jane Doe": ["Jane", "Ms. Doe", "Bob's mother"]
}}
"""

theme_prompt = """
You are given a list of themes with some duplicates or alternate names.
Your task is to group all the alternate names for each unique theme under a single name, using the descriptions as guidance.
Your goal is to have as few themes as possible while still capturing all the different ways the themes are described.

First create a new theme list by removing all duplicates and alternate names and only keeping the best description for each theme.

Then, output a JSON dictionary where the key is each theme in your new list and the value is a list of all possible alternate names. 
All names should be taken directly from the original theme list and should only appear once in the output.

For example:
{{
    "Memory": ["Remembering the past", "Recollection", "Nostalgia"],
    "Friendship and Loyalty": ["Friendship and Companionship", "Making Friends", "Bonds of Friendship", "Loyalty and Friendship"]
}}
"""

location_prompt = """
You are given a list of locations with some duplicates or alternate names.
Your task is to group all the alternate or similiar names for each unique location under a single location name.

First create a new location list by removing all duplicates and alternate names and only keeping the full name for each location.
If one location is inside another or nearby to it, only keep the larger location (e.g., "Bob's bedroom", "Bob's garden", and "Bob Smith's castle" can be listed under "Bob's house").
Then, output a JSON dictionary where the key is each location in your new list and the value is a list of all possible alternate location names.
All locations should be taken directly from the original location list and should only appear once in the output dict.

For example:
{{
    "Los Angeles": ["LA", "City of Angels", "Cafe in Los Angeles"],
    "Bob's house": ["Bob's bedroom", "Bob Smith's garden", "Bob's castle"]
}}
"""

if analysis_type == "theme":
    prompt = theme_prompt

# if using llm2 (claude)
prompt += "Just output the JSON dictionary as the final result without any additional information."
location_prompt += "Just output the JSON dictionary as the final result without any additional information."

prompt_template = ChatPromptTemplate.from_messages([("system", prompt), ("human", "{input}")])
location_prompt_template = ChatPromptTemplate.from_messages([("system", location_prompt), ("human", "{input}")])

In [223]:
# Create the LLM chain with the prompt template and model

# Run the chain by passing the characters and JSON data as inputs
if analysis_type == "theme":
    final_prompt = prompt_template.invoke(f"theme list: {all_characters}")
else:
    final_prompt = prompt_template.invoke(f"character list: {all_characters}")
# character_llm = llm.with_structured_output(None, method="json_mode") if model_type == "gpt" else llm
character_llm = llm2
# character_llm = llm
output = character_llm.invoke(final_prompt)


In [224]:
output

AIMessage(content='{\n    "Optimism": ["Philosophical Optimism", "Optimism vs. Reality"],\n    "Social Status": ["Class and Status"],\n    "Innocence": ["Innocence Shattered", "Loss of Innocence", "Destruction of Innocence"],\n    "Curiosity for Knowledge": ["Curiosity", "Curiosity vs. Apathy"],\n    "Romantic Aspirations": ["Love and Desire"],\n    "Forbidden Love": [],\n    "Social Rejection": [],\n    "Family Conflict": [],\n    "Despair": ["Despair and Anguish", "Despair and Rejection", "Despair and Horror", "Despair and Danger"],\n    "Isolation": [],\n    "Suffering": ["Human Suffering", "Suffering and Punishment", "Suffering and Recovery", "Suffering and Misery", "Suffering Comparison"],\n    "Generosity": ["Compassion and Generosity"],\n    "Community Support": [],\n    "Love and Loyalty": ["Love", "Loyalty"],\n    "Manipulation of Power": ["Power Dynamics"],\n    "Heroism and Glory": ["Heroism"],\n    "Punishment and Discipline": [],\n    "Improvement and Progress": [],\n    "

In [225]:
# if model_type == "claude":
output_formatted = output.content
# if output stop_reason = "max_tokens",cut off at last ']' character and add '}'
if output_formatted[-1] != "}":
    last_bracket_ind = output_formatted.rfind("]")
    output_formatted = output_formatted[:last_bracket_ind + 1] + "}"
    # print(output_formatted)
# turn into json
output = json.loads(output_formatted)

In [226]:
output

{'Optimism': ['Philosophical Optimism', 'Optimism vs. Reality'],
 'Social Status': ['Class and Status'],
 'Innocence': ['Innocence Shattered',
  'Loss of Innocence',
  'Destruction of Innocence'],
 'Curiosity for Knowledge': ['Curiosity', 'Curiosity vs. Apathy'],
 'Romantic Aspirations': ['Love and Desire'],
 'Forbidden Love': [],
 'Social Rejection': [],
 'Family Conflict': [],
 'Despair': ['Despair and Anguish',
  'Despair and Rejection',
  'Despair and Horror',
  'Despair and Danger'],
 'Isolation': [],
 'Suffering': ['Human Suffering',
  'Suffering and Punishment',
  'Suffering and Recovery',
  'Suffering and Misery',
  'Suffering Comparison'],
 'Generosity': ['Compassion and Generosity'],
 'Community Support': [],
 'Love and Loyalty': ['Love', 'Loyalty'],
 'Manipulation of Power': ['Power Dynamics'],
 'Heroism and Glory': ['Heroism'],
 'Punishment and Discipline': [],
 'Improvement and Progress': [],
 'Comradeship and Recognition': [],
 'Loss of Freedom': [],
 'Clemency and Forgiv

In [227]:
# Create the LLM chain with the prompt template and model
if not scene_details_exist:
    # Run the chain by passing the characters and JSON data as inputs
    final_location_prompt = location_prompt_template.invoke(f"location list: {all_locations}")
    # location_llm = llm.with_structured_output(None, method="json_mode") if model_type == "gpt" else llm
    location_llm = llm2
    # location_llm = llm
    location_output = location_llm.invoke(final_location_prompt)

In [228]:
if not scene_details_exist:
    print(location_output)

In [229]:
if not scene_details_exist:
    location_output_formatted = location_output.content
    # turn into json
    location_output = json.loads(location_output_formatted)

In [230]:
if not scene_details_exist: 
    print(location_output)

In [231]:
# Function to find the correct key based on a search string, ensuring whole-word matches
def find_key_from_alias(text, character_dict):
    for key, aliases in character_dict.items():
        for alias in sorted(aliases, key=len, reverse=True):
            # Match the alias as a whole word, and ensure no partial matches within other words
            pattern = rf'\b{re.escape(alias)}\b(?!\'s)'  # (?!\'s) ensures no match for possessives like Mitsuha's
            if re.search(pattern, text):
                return key  # Return the key if the alias is found as a whole word
    return text  # Return the original text if no match is found

In [232]:
# Load the scenes
with open(f"json/{story_name}/all.json") as f:
    scenes = json.load(f)

# Capitalize output and location output
capitalized_output = {string.capwords(key): [string.capwords(name) for name in value] for key, value in output.items()}

if not scene_details_exist:
    capitalized_location_output = {string.capwords(key): [string.capwords(name) for name in value] for key, value in location_output.items()}

# Process each scene
for scene in scenes:
    # Process characters
    for character in scene["characters"]:
        char = string.capwords(character["name"])
        found_char = False

        # Check if the character's name is a key in the dictionary
        if char in capitalized_output:
            found_char = True
            character["name"] = char  # Capitalize the character name
        
        # If not found in the dictionary, check the aliases
        if not found_char:
            # Find the right key for the alias (if exists)
            matched_key = find_key_from_alias(char, capitalized_output)
            if matched_key != char:
                character["name"] = matched_key
                found_char = True
        if not found_char:
            character["name"] = char

    if not scene_details_exist:
        # Process location
        loc = string.capwords(scene["location"])
        found_loc = False

        # Check if the location is a key in the location dictionary
        if loc in capitalized_location_output:
            found_loc = True
            scene["location"] = loc  # Capitalize the location

        # If not found in the dictionary, check the aliases
        if not found_loc:
            matched_key = find_key_from_alias(loc, capitalized_location_output)
            if matched_key != loc:
                scene["location"] = matched_key
                found_loc = True
        if not found_loc:
            scene["location"] = loc

# Save the updated scenes
with open(f"json/{story_name}/all.json", "w") as f:
    json.dump(scenes, f, indent=4)

In [233]:
# Remove duplicate characters in a scene and rank characters
# Also rename duplicate scene names

# Load the scenes
with open(f"json/{story_name}/all.json") as f:
    scenes = json.load(f)

scene_name_counts = {}

for j, scene in enumerate(scenes):
    scene_name = scene["title"]
    if scene_name in scene_name_counts:
        # update scene name if it is a duplicate
        scene_name_counts[scene_name] += 1
        scene["title"] = f"{scene_name} {scene_name_counts[scene_name]}"
    else:
        scene_name_counts[scene_name] = 1

    # remove duplicate characters
    unique_characters = []
    new_character_info = []
    for character in scene["characters"]:
        char_name = character["name"]
        if char_name not in unique_characters:
            unique_characters.append(char_name)
            new_character_info.append(character)
    scene["characters"] = new_character_info

    # now extract character importances and rank
    character_importances = []
    for k, character in enumerate(scene["characters"]):
        character_importances.append((k, character["importance"]))
    # sort character importances
    sorted_character_importances = sorted(character_importances, key=lambda x: x[1], reverse=True)
    # add importance_rank to each character
    for k, (l, _) in enumerate(sorted_character_importances):
        scenes[j]["characters"][l]["importance_rank"] = k+1

# Save the updated scenes
with open(f"json/{story_name}/all.json", "w") as f:
    json.dump(scenes, f, indent=4)

In [234]:
# now list all characters and locations again
# collect all unique characters and locations for each chapter
all_characters_new = {}
all_locations_new = []
with open(f"json/{story_name}/all.json") as f:
    scenes = json.load(f)
for scene in scenes:
    for character in scene["characters"]:
        char = character["name"]
        if char not in all_characters_new:
            all_characters_new[char] = 1
        else:
            all_characters_new[char] += 1
    location = scene["location"]
    if location not in all_locations_new:
        all_locations_new.append(location)

In [235]:
len(all_characters_new), len(all_locations_new)

(199, 90)

In [236]:
all_characters_new 

{'Optimism': 8,
 'Social Status': 6,
 'Innocence': 5,
 'Curiosity For Knowledge': 5,
 'Romantic Aspirations': 2,
 'Forbidden Love': 1,
 'Social Rejection': 1,
 'Family Conflict': 1,
 'Despair': 7,
 'Isolation': 3,
 'Suffering': 12,
 'Generosity': 3,
 'Community Support': 1,
 'Love And Loyalty': 10,
 'Manipulation Of Power': 2,
 'Heroism And Glory': 2,
 'Punishment And Discipline': 1,
 'Improvement And Progress': 1,
 'Comradeship And Recognition': 1,
 'Loss Of Freedom': 1,
 'Clemency And Forgiveness': 1,
 'Healing And Recovery': 1,
 'War And Conflict': 5,
 'Survival': 10,
 'Disillusionment': 8,
 'Hope Vs. Reality': 1,
 'Charity And Almsgiving': 3,
 'Inhumanity And Cruelty': 1,
 'Compassion And Horror': 1,
 'Loss': 8,
 'Violence And Suffering': 5,
 'Corruption Of Humanity': 1,
 "Nature's Fury": 2,
 'Survival And Death': 1,
 'Futility Of Reason': 1,
 'Chaos And Destruction': 1,
 'Original Sin': 1,
 'Liberty Vs Necessity': 1,
 'Philosophical Debate': 4,
 'Religious Hypocrisy': 4,
 'Absurdi

In [237]:
all_locations_new

["Baron Of Thunder-ten-tronckh's Castle",
 'Castle Park',
 'Behind The Screen',
 'Field Near Waldberghofftrarbk-dikdorff',
 'Dining Table',
 'Regiment Barracks',
 'Training Grounds',
 'Dungeon',
 'Kingdom Of The Bulgarians',
 'Battlefield',
 'Holland',
 'Public Square',
 "James' Home",
 'City Street',
 "Anabaptist's Stable",
 'Ship To Lisbon',
 'Inquisition Chamber',
 'Lisbon',
 'Unknown Location',
 'Decayed House',
 'Lonely House In The Country',
 "Cunegonde's House",
 "Cunegonde's Story",
 'On The Sofa',
 "Cunegonde's Feet",
 'Stable',
 'Public Inn In Avacena',
 'On The Voyage',
 "Old Woman's Home",
 'Massa Carara',
 'On Board The Pirate Vessel',
 'Morocco',
 'Orange Tree By Rivulet',
 'Algiers',
 'Fort On Palus Méotides',
 'On Board A Ship',
 'Buenos Ayres',
 'On The Road To Paraguay',
 "Commandant's Arbour",
 'Chapel Of The Jesuits',
 'Beautiful Meadow',
 'Wood Near Buenos Ayres',
 'Frontiers Of The Oreillons',
 'Near A Little River',
 'First Village',
 "Landlord's Residence",
 'Pl

### double check character quotes

In [238]:
def is_direct_quote(quote, scene_text):
    scene_text = scene_text.replace("\n", " ").replace("_", "").lower()
    scene_text = re.sub(r'[^\w\s]', '', scene_text) # remove punctuation

    quote = quote.replace("_", "").lower()
    quote = re.sub(r'[^\w\s]', '', quote) # remove punctuation

    return quote in scene_text

In [239]:
wrong_quotes = []

# load in scenes
with open(f"json/{story_name}/all.json") as f:
    scenes = json.load(f)

    for scene in scenes:
        scene_text = scene["text"]
        scene_name = scene["title"]
        scene_chapter = scene["chapter"]
        for character in scene["characters"]:
            char_name = character["name"]
            quote = character["quote"]
            emotion = character["emotion"]
            if "No quote available" not in character["quote"] and not is_direct_quote(quote, scene_text):
                wrong_quotes.append({"scene": scene_name, "chapter": scene_chapter, "character": char_name, "emotion": emotion, "quote": quote})

In [240]:
len(wrong_quotes), wrong_quotes

(27,
 [{'scene': 'Forced into the Regiment',
   'chapter': 'II WHAT BECAME OF CANDIDE AMONG THE BULGARIANS.',
   'character': 'Love And Loyalty',
   'emotion': 'Conflicted affection',
   'quote': 'Oh yes, I deeply love Miss Cunegonde.'},
  {'scene': 'Shipwreck Chaos',
   'chapter': 'V TEMPEST, SHIPWRECK, EARTHQUAKE, AND WHAT BECAME OF DOCTOR PANGLOSS, CANDIDE, AND JAMES THE ANABAPTIST.',
   'character': 'Futility Of Reason',
   'emotion': 'Absurdity',
   'quote': 'he was prevented by the philosopher Pangloss, who demonstrated to him that the Bay of Lisbon had been made on purpose for the Anabaptist to be drowned.'},
  {'scene': 'Philosophical Debate',
   'chapter': 'V TEMPEST, SHIPWRECK, EARTHQUAKE, AND WHAT BECAME OF DOCTOR PANGLOSS, CANDIDE, AND JAMES THE ANABAPTIST.',
   'character': 'Liberty Vs Necessity',
   'emotion': 'Philosophical',
   'quote': 'Sir, you do not then believe in liberty?'},
  {'scene': "Candide's Recovery",
   'chapter': 'VII HOW THE OLD WOMAN TOOK CARE OF CANDID

In [241]:
# Pydantic
class CharacterExplanation(BaseModel):
    """Get information about a character's quotes in the story."""
    number: int = Field(description="Quote number")
    character: str = Field(description="Character name")
    exp: str = Field(description="Brief, 1-sentence explanation of why the character is feeling the emotion based on the quote. Include relevant context from the quote but do not directly refer to it.")

class CharacterExplanationList(BaseModel):
    """Get overall information about the story."""
    characters: list[CharacterExplanation] = Field(description="List of character emotion explanations")

class ThemeExplanation(BaseModel):
    """Get information about a theme's quotes in the story."""
    number: int = Field(description="Quote number")
    theme: str = Field(description="Theme name")
    exp: str = Field(description="Brief, 1-sentence explanation of how the theme reflects to the emotion. Include relevant context from the quote but do not directly refer to it.")

class ThemeExplanationList(BaseModel):
    """Get overall information about the story."""
    themes: list[ThemeExplanation] = Field(description="List of theme emotion explanations")

char_exp_llm = llm.with_structured_output(CharacterExplanationList)
theme_exp_llm = llm.with_structured_output(ThemeExplanationList)

In [242]:
# async function to get character explanations
async def get_character_explanations_async(quotes):
    print(f"Running one batch...")

    quotes_formatted = [{"number": i, "character": quote["character"], "emotion": quote["emotion"], "quote": quote["quote"]} for i, quote in enumerate(quotes)]

    prompt = f"""
             character quotes: {quotes_formatted}

             your output should contain exactly the same {len(quotes_formatted)} characters as the input.
             """
    
    theme_prompt = f"""
                theme quotes: {quotes_formatted}

                your output should contain exactly the same {len(quotes_formatted)} themes as the input.
                """
    
    character_explanations = await char_exp_llm.ainvoke(prompt) if analysis_type == "character" else await theme_exp_llm.ainvoke(theme_prompt)
    explanations = character_explanations.characters if analysis_type == "character" else character_explanations.themes

    for explanation in explanations:
        num = explanation.number
        exp = explanation.exp
        char = explanation.character if analysis_type == "character" else explanation.theme
        if (char != quotes[num]["character"]):
            print("Error: character name does not match")
        quotes[num]["exp"] = exp
    
    return quotes

In [243]:
# run async function in parallel
max_quotes_per_batch = 20
tasks = [semaphore_wrapper(get_character_explanations_async, wrong_quotes[i:i+max_quotes_per_batch]) for i in range(0, len(wrong_quotes), max_quotes_per_batch)]
final_quotes = await asyncio.gather(*tasks)
print("Done")

Running one batch...
Running one batch...
Done


In [244]:
final_quotes

[[{'scene': 'Forced into the Regiment',
   'chapter': 'II WHAT BECAME OF CANDIDE AMONG THE BULGARIANS.',
   'character': 'Love And Loyalty',
   'emotion': 'Conflicted affection',
   'quote': 'Oh yes, I deeply love Miss Cunegonde.',
   'exp': "The character's declaration of deep love reflects a complex emotional state where affection is intertwined with conflicting feelings about loyalty."},
  {'scene': 'Shipwreck Chaos',
   'chapter': 'V TEMPEST, SHIPWRECK, EARTHQUAKE, AND WHAT BECAME OF DOCTOR PANGLOSS, CANDIDE, AND JAMES THE ANABAPTIST.',
   'character': 'Futility Of Reason',
   'emotion': 'Absurdity',
   'quote': 'he was prevented by the philosopher Pangloss, who demonstrated to him that the Bay of Lisbon had been made on purpose for the Anabaptist to be drowned.',
   'exp': "The absurdity of the situation highlights the character's struggle to find meaning in a world where reason seems futile."},
  {'scene': 'Philosophical Debate',
   'chapter': 'V TEMPEST, SHIPWRECK, EARTHQUAKE, A

In [245]:
# replace wrong quotes with explanations
for i, quotes in enumerate(final_quotes):
    for quote in quotes:
        s = quote["scene"]
        chapter = quote["chapter"]
        char = quote["character"]

        # find scene
        for scene in scenes:
            scene_name = scene["title"]
            if scene_name == s and scene["chapter"] == chapter:
                for character in scene["characters"]:
                    if character["name"] == char:
                        character["quote"] = "No quote available"
                        character["fake_quote"] = quote["exp"]
                        print("Fixed quote for", char, "in", scene_name)
                        break

Fixed quote for Love And Loyalty in Forced into the Regiment
Fixed quote for Futility Of Reason in Shipwreck Chaos
Fixed quote for Liberty Vs Necessity in Philosophical Debate
Fixed quote for Charity And Almsgiving in Candide's Recovery
Fixed quote for Loss in Reunion with Cunegonde
Fixed quote for Hope And Resilience in Reunion with Cunegonde
Fixed quote for Conflict And Oppression in Supper Table
Fixed quote for Violence And Suffering in Battle for Survival
Fixed quote for Survival in Fainting Under Stress
Fixed quote for Desire And Marriage in Arrival at Buenos Ayres
Fixed quote for Cunning And Deception in Capture by Oreillons
Fixed quote for Childhood Innocence in Playing with the Children
Fixed quote for Value Of Wealth in Playing with the Children
Fixed quote for Hospitality in Dinner and Discovery
Fixed quote for Cultural Contrast in Reception by the King
Fixed quote for Obstacles And Challenges in King's Advice
Fixed quote for Cultural Contrast in Preparation for Departure
Fix

In [246]:
# replace in json
with open(f"json/{story_name}/all.json", "w") as f:
    json.dump(scenes, f, indent=4)

## produce chapter summaries

In [247]:
# Pydantic
class ChapterSummary(BaseModel):
    """Summarize a chapter based on scenes."""
    summary: str = Field(description="A brief, 1-line summary of the chapter")

chapter_llm = llm.with_structured_output(ChapterSummary)

In [248]:
class CharacterLink(BaseModel):
    """Summarize the interaction between two characters in this chapter."""
    char1: str = Field(description="Name of the first character")
    char2: str = Field(description="Name of the second character")
    interaction: str = Field(description="Brief description of the interaction between the two characters")

class CharacterLinks(BaseModel):
    """Summarize all interactions between characters in a chapter."""
    links: list[CharacterLink] = Field(description="List of interactions between pairs of characters in the scene")

class ThemeLink(BaseModel):
    """Summarize the interaction between two themes in this chapter."""
    char1: str = Field(description="Name of the first theme")
    char2: str = Field(description="Name of the second theme")
    interaction: str = Field(description="Brief description of the interaction between the two themes")

class ThemeLinks(BaseModel):
    """Summarize all interactions between themes in a chapter."""
    links: list[ThemeLink] = Field(description="List of interactions between pairs of themes in the scene")

character_links_llm = llm.with_structured_output(CharacterLinks)
theme_links_llm = llm.with_structured_output(ThemeLinks)

In [249]:
# remove any markers without any scenes
if not scene_details_exist:
    markers_to_remove = []
    for i, marker in enumerate(markers):
        with open(f"json/{story_name}/{marker}.json") as f:
            scenes = json.load(f)
        if len(scenes) == 0:
            markers_to_remove.append(marker)

    for marker in markers_to_remove:
        markers.remove(marker)

In [250]:
async def read_json_async(file_path):
    """Asynchronously read a JSON file."""
    async with aiofiles.open(file_path, mode="r") as f:
        data = await f.read()
        return json.loads(data)
    
async def read_text_async(file_path):
    """Asynchronously read a text file."""
    async with aiofiles.open(file_path, mode="r") as f:
        return await f.read()
    
async def process_chapter_summary_async(og_story_name, marker, scenes):
    """Process the summary for a single chapter."""
    chapter_file = f"chapters/{og_story_name}/{marker}.txt"

    chapter_scenes = [scene for scene in scenes if scene["chapter"] == marker]

    # Read chapter text file asynchronously
    chapter_text = await read_text_async(chapter_file)

    # Create scene summaries string
    scene_summaries = " ".join([scene["summary"] for scene in chapter_scenes])

    # Get summary from LLM
    summary = await chapter_llm.ainvoke(scene_summaries)
    print(f"Chapter {marker} summary generated.")

    # Create character links 
    links = []
    for scene in chapter_scenes:
        characters = scene["characters"]
        for j in range(len(characters)):
            char1 = characters[j]
            char1_name = char1["name"]
            for k in range(j+1, len(characters)):
                char2 = characters[k]
                char2_name = char2["name"]
                
                # check if link already exists
                existing_link = next((link for link in links if (link["source"] == char1_name and link["target"] == char2_name)
                                        or (link["source"] == char2_name and link["target"] == char1_name)), None)
                if existing_link:
                    # increment value if link already exists
                    existing_link["value"] += 1
                else:
                    # add new link
                    links.append({"source": char1_name, "target": char2_name, "value": 1})
    
    # Create character links summary
    character_links = [{"char1": link["source"], "char2": link["target"]} for link in links]
    link_prompt = f"""
                    Character pairs: {character_links}
                    Scene summaries: {scene_summaries}

                    Based on the scene summaries above, summarize how each pair of characters interacts in this chapter.
                    Your output should contain exactly the same {len(character_links)} pairs as in the original list.
                    """
    theme_link_prompt = f"""
                    Theme pairs: {character_links}
                    Scene summaries: {scene_summaries}

                    Based on the scene summaries above, summarize how each pair of themes interact in this chapter.
                    Your output should contain exactly the same {len(character_links)} pairs as in the original list.
                    """

    # Get character links from LLM
    character_links_summary = await character_links_llm.ainvoke(link_prompt) if analysis_type == "character" else await theme_links_llm.ainvoke(theme_link_prompt)
    character_links_list = character_links_summary.links

    print(f"Character links for Chapter {marker} generated.")

    # Add interactions to character links using the LLM output
    for llm_link in character_links_list:
        char1_name = llm_link.char1
        char2_name = llm_link.char2
        for link in links:
            if (link["source"] == char1_name and link["target"] == char2_name) or (link["source"] == char2_name and link["target"] == char1_name):
                link["interaction"] = llm_link.interaction

    chapter_summary = {
        "chapter": marker,
        "scenes": len(chapter_scenes),
        "length": len(chapter_text),
        "num_lines": len(chapter_text.split("\n")),
        "summary": summary.summary,
        "importance": [],
        "conflict": [],
        "locations": {},
        "characters": {},
        "links": links
    }

    return chapter_summary

async def generate_chapter_summaries_async(story_name, og_story_name, markers):
    """Generate summaries for all chapters in parallel."""
    # Read all.json file asynchronously
    scenes = await read_json_async(f"json/{story_name}/all.json")

    # Processs chapters in parallel
    chapter_summaries = await asyncio.gather(*[process_chapter_summary_async(og_story_name, marker, scenes) for marker in markers])
    print("All chapter summaries generated.")

    # Aggregate character and location information
    for chapter_summary in chapter_summaries:
        chapter = chapter_summary["chapter"]

        chapter_scenes = [scene for scene in scenes if scene["chapter"] == chapter]
    
        for scene in chapter_scenes:
            # add location to chapter summary
            location = scene["location"]
            chapter_summary_locations = chapter_summary["locations"]
            if location not in chapter_summary_locations:
                chapter_summary_locations[location] = 1
            else:
                chapter_summary_locations[location] += 1
            # add importance to chapter summary
            importance = scene["importance"]
            chapter_summary["importance"].append(importance)
            # add conflict to chapter summary
            conflict = scene["conflict"]
            chapter_summary["conflict"].append(conflict)
            # add characters to chapter summary
            characters = scene["characters"]
            for character in characters:
                char = character["name"]
                chapter_summary_characters = chapter_summary["characters"]
                if char not in chapter_summary_characters:
                    chapter_summary_characters[char] = 1
                else:
                    chapter_summary_characters[char] += 1

    # now average importance and conflict for each chapter
    for chapter_summary in chapter_summaries:
        if len(chapter_summary["importance"]) == 0:
            print(f"Chapter {chapter_summary['chapter']} has no scenes.")
            continue
        chapter_summary["importance"] = round(sum(chapter_summary["importance"]) / len(chapter_summary["importance"]), 2)
        chapter_summary["conflict"] = round(sum(chapter_summary["conflict"]) / len(chapter_summary["conflict"]), 2)

    print("Importances", [chapter_summary["importance"] for chapter_summary in chapter_summaries])
    print("Conflicts", [chapter_summary["conflict"] for chapter_summary in chapter_summaries])

    # rank each chapter by importance and conflict
    importances = []
    conflicts = []
    for chapter_summary in chapter_summaries:
        importances.append((chapter_summary["chapter"], chapter_summary["importance"]))
        conflicts.append((chapter_summary["chapter"], chapter_summary["conflict"]))
        
    # sort importances
    sorted_importances = sorted(importances, key=lambda x: x[1], reverse=True)
    # add importance_rank to each chaptera
    for k, (j, _) in enumerate(sorted_importances):
        chapter_summaries[markers.index(j)]["importance_rank"] = k+1

    # sort conflicts
    sorted_conflicts = sorted(conflicts, key=lambda x: x[1], reverse=True)
    # add conflict_rank to each chapter
    for k, (j, _) in enumerate(sorted_conflicts):
        chapter_summaries[markers.index(j)]["conflict_rank"] = k+1

    return chapter_summaries

In [251]:
async def process_chapter_summary_async_characters_only(chapter_summary, marker, scenes):
    """Process the summary for a single chapter."""

    chapter_scenes = [scene for scene in scenes if scene["chapter"] == marker]

    # Create scene summaries string
    scene_summaries = " ".join([scene["summary"] for scene in chapter_scenes])

    # Create character links 
    links = []
    for scene in chapter_scenes:
        characters = scene["characters"]
        for j in range(len(characters)):
            char1 = characters[j]
            char1_name = char1["name"]
            for k in range(j+1, len(characters)):
                char2 = characters[k]
                char2_name = char2["name"]
                
                # check if link already exists
                existing_link = next((link for link in links if (link["source"] == char1_name and link["target"] == char2_name)
                                        or (link["source"] == char2_name and link["target"] == char1_name)), None)
                if existing_link:
                    # increment value if link already exists
                    existing_link["value"] += 1
                else:
                    # add new link
                    links.append({"source": char1_name, "target": char2_name, "value": 1})
    
    # Create character links summary
    character_links = [{"char1": link["source"], "char2": link["target"]} for link in links]
    link_prompt = f"""
                    Character pairs: {character_links}
                    Scene summaries: {scene_summaries}

                    Based on the scene summaries above, summarize how each pair of characters interacts in this chapter.
                    Your output should contain exactly the same {len(character_links)} pairs as in the original list.
                    """
    theme_link_prompt = f"""
                    Theme pairs: {character_links}
                    Scene summaries: {scene_summaries}

                    Based on the scene summaries above, summarize how each pair of themes interact in this chapter.
                    Your output should contain exactly the same {len(character_links)} pairs as in the original list.
                    """

    # Get character links from LLM
    character_links_summary = await character_links_llm.ainvoke(link_prompt) if analysis_type == "character" else await theme_links_llm.ainvoke(theme_link_prompt)
    character_links_list = character_links_summary.links

    print(f"Character links for Chapter {marker} generated.")

    # Add interactions to character links using the LLM output
    for llm_link in character_links_list:
        char1_name = llm_link.char1
        char2_name = llm_link.char2
        for link in links:
            if (link["source"] == char1_name and link["target"] == char2_name) or (link["source"] == char2_name and link["target"] == char1_name):
                link["interaction"] = llm_link.interaction

    chapter_summary["links"] = links
    chapter_summary["characters"] = {}

    return chapter_summary

async def generate_chapter_summaries_async_characters_only(story_name, og_story_name, markers):
    """Generate summaries for all chapters in parallel."""
    print(f"Updating existing chapter information for {analysis_type} data...")
    # Read all.json file asynchronously
    scenes = await read_json_async(f"json/{story_name}/all.json")

    with open(f"json/{og_story_name}/final_data.json") as f:
        chapters_old = json.load(f)["chapters"]
    chapter_summaries = await asyncio.gather(*[process_chapter_summary_async_characters_only(chapter_summary, marker, scenes) for chapter_summary, marker in zip(chapters_old, markers)])

    # Aggregate character and location information
    for chapter_summary in chapter_summaries:
        chapter = chapter_summary["chapter"]

        chapter_scenes = [scene for scene in scenes if scene["chapter"] == chapter]
    
        for scene in chapter_scenes:
            # add characters to chapter summary
            characters = scene["characters"]
            for character in characters:
                char = character["name"]
                chapter_summary_characters = chapter_summary["characters"]
                if char not in chapter_summary_characters:
                    chapter_summary_characters[char] = 1
                else:
                    chapter_summary_characters[char] += 1

    return chapter_summaries

In [252]:
# create summary json of each chapter
chapter_summaries = await generate_chapter_summaries_async(story_name, og_story_name, markers) if not scene_details_exist else await generate_chapter_summaries_async_characters_only(story_name, og_story_name, markers)

Updating existing chapter information for theme data...
Character links for Chapter XXI CANDIDE AND MARTIN, REASONING, DRAW NEAR THE COAST OF FRANCE. generated.
Character links for Chapter VII HOW THE OLD WOMAN TOOK CARE OF CANDIDE, AND HOW HE FOUND THE OBJECT HE LOVED. generated.
Character links for Chapter XXIX HOW CANDIDE FOUND CUNEGONDE AND THE OLD WOMAN AGAIN. generated.
Character links for Chapter VI HOW THE PORTUGUESE MADE A BEAUTIFUL AUTO-DA-FÉ, TO PREVENT ANY FURTHER EARTHQUAKES; AND HOW CANDIDE WAS PUBLICLY WHIPPED. generated.
Character links for Chapter XXIII CANDIDE AND MARTIN TOUCHED UPON THE COAST OF ENGLAND, AND WHAT THEY SAW THERE. generated.
Character links for Chapter III HOW CANDIDE MADE HIS ESCAPE FROM THE BULGARIANS, AND WHAT AFTERWARDS BECAME OF HIM. generated.
Character links for Chapter XIII HOW CANDIDE WAS FORCED AWAY FROM HIS FAIR CUNEGONDE AND THE OLD WOMAN. generated.
Character links for Chapter XVI ADVENTURES OF THE TWO TRAVELLERS, WITH TWO GIRLS, TWO MONKE

In [253]:
chapter_summaries

[{'chapter': 'I HOW CANDIDE WAS BROUGHT UP IN A MAGNIFICENT CASTLE, AND HOW HE WAS EXPELLED THENCE.',
  'scenes': 3,
  'length': 5095,
  'num_lines': 69,
  'summary': "Candide's sheltered life in the Baron's castle is disrupted when he and Cunegonde share a forbidden kiss, while Pangloss's philosophical teachings unfold.",
  'importance': 0.8,
  'conflict': 0.43,
  'locations': {"Baron Of Thunder-ten-tronckh's Castle": 1,
   'Castle Park': 1,
   'Behind The Screen': 1},
  'characters': {'Optimism': 1,
   'Social Status': 1,
   'Innocence': 1,
   'Curiosity For Knowledge': 1,
   'Romantic Aspirations': 1,
   'Forbidden Love': 1,
   'Social Rejection': 1,
   'Family Conflict': 1},
  'links': [{'source': 'Optimism',
    'target': 'Social Status',
    'value': 1,
    'interaction': "Candide's optimistic outlook is contrasted with the rigid social hierarchy of the Baron's castle, highlighting the disparity between his hopeful nature and the constraints of social status."},
   {'source': 'Op

In [254]:
# read in scenes from all_json
with open(f"json/{story_name}/all.json") as f:
    scenes = json.load(f)

In [255]:
all_json = {}
all_json["title"] = story_info["title"]
all_json["type"] = story_info["type"]
if (all_json["type"] == "Movie"):
    all_json["director"] = story_info["director"]
else:
    all_json["author"] = story_info["author"]
all_json["year"] = story_info["year"]
if "url" in story_info:
    all_json["url"] = story_info["url"]
if "image" in story_info:
    all_json["image"] = story_info["image"]
all_json["num_chapters"] = len(chapters)
all_json["num_scenes"] = len(scenes)
all_json["num_characters"] = len(all_characters_new)
all_json["num_locations"] = len(all_locations_new)
all_json["chapters"] = chapter_summaries
all_json["scenes"] = scenes

## produce character summaries

In [256]:
# Pydantic
class CharacterInfo(BaseModel):
    """Get information about a character in the story."""
    name: str = Field(description="Name of character.")
    quote: str = Field(description="Direct quote from the story that the character says or that describes them.")
    group: str = Field(description="Group that this character belongs to (e.g., main characters, upper class, talking animals). There should be at least 2 different groups of characters.")
    # color: str = Field(description="Unique aesthetic RGB color string that represents this character (e.g., rgb(118, 185, 71)). Every character should have a different color. Don't use white and make sure the color is visible against a white background.")
    # explanation: str = Field(description="Explanation why this color was chosen for this character.")

class LegendThemeInfo(BaseModel):
    """Get information about a theme in the story."""
    name: str = Field(description="Name of theme.")
    quote: str = Field(description="Direct quote from the story that describes this theme.")
    group: Optional[str] = Field(description="Group that this theme belongs to (e.g., Family, Social Themes, Emotions). There should be at least 2 different groups of themes. Make sure all groups are distinct and don't overlap with each other.")

class CharacterList(BaseModel):
    """Get overall information about the story."""
    # characterlist: list[CharacterInfo] = Field(description="List of character details. Make sure there is exactly one entry per character in the provided list and no additional characters are added. Choose a different color for each character.")
    characterlist: list[CharacterInfo] = Field(description="List of character details. Make sure there is exactly one entry per character in the provided list and no additional characters are added.")

class LegendThemeList(BaseModel):
     """Get overall information about the story."""
     themelist: list[LegendThemeInfo] = Field(description="List of theme details. Make sure there is exactly one entry per theme in the provided list and no additional themes are added.")

class ColorInfo(BaseModel):
    """Pick a color + explain why for each character"""
    name: str = Field(description="Name of character.")
    color: str = Field(description="Unique aesthetic RGB color string that represents this character (e.g., rgb(118, 185, 71)). Every character should have a different color. Don't use white and make sure the color is visible against a white background.")
    explanation: str = Field(description="Explanation why this color was chosen for this character.")

class ThemeColorInfo(BaseModel):
    """Pick a color + explain why for each theme"""
    name: str = Field(description="Name of theme.")
    color: str = Field(description="Unique aesthetic RGB color string that represents this theme (e.g., rgb(118, 185, 71)). Every theme should have a different color. Don't use white and make sure the color is visible against a white background.")
    explanation: str = Field(description="Explanation why this color was chosen for this theme.")

class ColorList(BaseModel):
    """List of color info for characters"""
    colorlist: list[ColorInfo] = Field(description="List of color details for each character. Make sure there is exactly one entry per character in the provided list and no additional characters are added. Choose a different color for each character.")

class ThemeColorList(BaseModel):
    """List of color info for themes"""
    colorlist: list[ThemeColorInfo] = Field(description="List of color details for each theme. Make sure there is exactly one entry per theme in the provided list and no additional themes are added. Choose a different color for each theme.")

if analysis_type == "theme":
    characterlist_llm = llm.with_structured_output(LegendThemeList)
    colorlist_llm = llm.with_structured_output(ThemeColorList)
else:
    characterlist_llm = llm.with_structured_output(CharacterList)
    colorlist_llm = llm.with_structured_output(ColorList)

In [257]:
# sort all_characters_new by count, descending
sorted_characters = dict(sorted(all_characters_new.items(), key=lambda item: item[1], reverse=True))
# take top 20 characters
top_characters = dict(list(sorted_characters.items())[:20])
top_characters

{'Suffering': 12,
 'Hope And Resilience': 12,
 'Love And Loyalty': 10,
 'Survival': 10,
 'Escape And Adventure': 10,
 'Optimism': 8,
 'Disillusionment': 8,
 'Loss': 8,
 'Despair': 7,
 'Absurdity Of Justice': 7,
 'Cultural Contrast': 7,
 'Social Status': 6,
 'Philosophical Reflection': 6,
 'Innocence': 5,
 'Curiosity For Knowledge': 5,
 'War And Conflict': 5,
 'Violence And Suffering': 5,
 'Wealth': 5,
 'Friendship And Loyalty': 5,
 'Philosophical Debate': 4}

In [258]:
character_arr = []
cur_arr = []
max_chars = 20

char_names = list(all_characters_new.keys())
# char_names = list(top_characters.keys())
for i, c in enumerate(char_names):
    cur_arr.append(c)
    if len(cur_arr) == max_chars or i == len(char_names) - 1:
        character_arr.append(cur_arr)
        cur_arr = []

for arr in character_arr:
    print(len(arr))


20
20
20
20
20
20
20
20
20
19


In [259]:
async def process_character_chunk(i, arr):
    """Process a single chunk of character assignment asynchronously."""
    invalid_input = True
    while invalid_input:
        try:
            # filter scenes to include only those with relevant characters
            new_scenes = [scene for scene in scenes if any(char["name"] in arr for char in scene["characters"])]

             # take middle 126000 characters of new_scenes
            new_scenes_str = json.dumps(new_scenes)
            new_scenes_str = new_scenes_str[:126000]

            # construct prompt
            prompt = f"""characters: {arr}
                         story info: {new_scenes_str}

                         your output should contain exactly the same {len(arr)} characters as in the original list.

                         remember to only use direct quotes from the text and not paraphrase or change the text in any way.
                         """

            # invoke model
            res = await characterlist_llm.ainvoke(prompt)

            print(f"Chunk {i} processed.")
            return res

        except Exception as e:
            print("Exception thrown. Trying again.")
            print("Error:", e)
            await asyncio.sleep(1) # wait for 1 second before trying again

async def process_all_characters():
    """Process all characters in parallel."""
    tasks = [process_character_chunk(i, arr) for i, arr in enumerate(character_arr)]
    results = await asyncio.gather(*tasks)
    return results

In [260]:
# Run the main function to process all characters
all_res = await process_all_characters()

Chunk 6 processed.
Chunk 9 processed.
Chunk 5 processed.
Chunk 8 processed.
Chunk 4 processed.
Chunk 3 processed.
Chunk 0 processed.
Chunk 2 processed.
Chunk 1 processed.
Chunk 7 processed.


In [261]:
# combine characterlists into one
characters = []
for res in all_res:
    if analysis_type == "theme":
        print(len(res.themelist))
        characters += res.themelist
    else:
        print(len(res.characterlist))
        characters += res.characterlist

len(characters)

20
20
20
20
20
20
20
20
20
19


199

In [262]:
character_names = [char.name for char in characters]
for name in character_names:
    if name not in char_names:
        print("not in orig list:", name)

need_to_add = []
for name in char_names:
    if name not in character_names:
        print("not in llm output:", name)
        need_to_add.append(name)

# find duplicate names
unique_names = []
duplicate_indices = []
for i, name in enumerate(character_names):
    if name not in unique_names:
        unique_names.append(name)
    else:
        print("duplicate:", name, i)
        duplicate_indices.append(i)

print("need_to_add", len(need_to_add))
print("duplicate_indices", len(duplicate_indices))


need_to_add 0
duplicate_indices 0


### group duplicate character groups

In [263]:
# group similar groups

group_prompt = """
You are given a list of groups with some duplicates or alternate names.
Your task is to group all the alternate names for each unique group under a single name.
Your goal is to have as few groups as possible while still capturing all the unique groups.

First create a new group list by removing all duplicates and alternate names and only keeping the best description for each group.

Then, output a JSON dictionary where the key is each group in your new list and the value is a list of all possible alternate names. 
All names should be taken directly from the original group list and should only appear once in the output.

For example:
{{
    "Main Characters": ["Main Characters", "Protagonists", "Heroes"],
    "Elders": ["Elders", "Family Authorities"],
    "Exploration": ["Exploration", "Adventure", "Discovery"],
    "Relationships": ["Relationships", "Friendships"]
}}
"""

# if using llm2 (claude)
group_prompt += "Just output the JSON dictionary as the final result without any additional information."

group_prompt_template = ChatPromptTemplate.from_messages([("system", group_prompt), ("human", "{input}")])

In [264]:
# get all unique groups
# groups = [c['group'] for c in characters]
groups = [c.group for c in characters]
unique_groups = list(set(groups))
len(unique_groups), unique_groups

(47,
 ['Providence',
  'Power Dynamics',
  'Emotions',
  'Suffering',
  'Power',
  'Social Themes',
  'Character Dynamics',
  'Cultural Themes',
  'Misery',
  'Personal Development Themes',
  'Identity',
  'Art and Perception',
  'Societal Themes',
  'Irony',
  'Moral Themes',
  'Philosophical Themes',
  'Survival',
  'Danger',
  'Social Structure',
  'Romantic Themes',
  'Desire',
  'Justice',
  'Utopian Themes',
  'Tragedy',
  'Adaptability',
  'Culture',
  'Deception',
  'Hope',
  'Escape',
  'Personal Identity',
  'Objectification',
  'Ambition',
  'Conflict Themes',
  'Emotional Themes',
  'Family Themes',
  'Respect',
  'Human Villainy',
  'Human Nature',
  'Intellectual Themes',
  'Religious Themes',
  'Historical Themes',
  'Heroic Themes',
  'Nature and Appreciation',
  'Hierarchy',
  'Conflict',
  'Economic Themes',
  'Political Themes'])

In [265]:
# Create the LLM chain with the prompt template and model

# Run the chain by passing the characters and JSON data as inputs
if analysis_type == "theme":
    final_prompt = group_prompt_template.invoke(f"list of groups for themes: {unique_groups}")
else:
    final_prompt = group_prompt_template.invoke(f"list of groups for characters: {unique_groups}")
group_llm = llm2
output = group_llm.invoke(final_prompt)

In [266]:
output

AIMessage(content='{\n    "Providence": ["Providence"],\n    "Power Dynamics": ["Power Dynamics", "Power", "Hierarchy"],\n    "Emotions": ["Emotions", "Emotional Themes"],\n    "Suffering": ["Suffering", "Misery"],\n    "Social Themes": ["Social Themes", "Societal Themes", "Social Structure"],\n    "Character Dynamics": ["Character Dynamics"],\n    "Cultural Themes": ["Cultural Themes", "Culture"],\n    "Personal Development": ["Personal Development Themes", "Adaptability"],\n    "Identity": ["Identity", "Personal Identity"],\n    "Art and Perception": ["Art and Perception"],\n    "Irony": ["Irony"],\n    "Moral Themes": ["Moral Themes"],\n    "Philosophical Themes": ["Philosophical Themes"],\n    "Survival": ["Survival"],\n    "Danger": ["Danger"],\n    "Romantic Themes": ["Romantic Themes"],\n    "Desire": ["Desire"],\n    "Justice": ["Justice"],\n    "Utopian Themes": ["Utopian Themes"],\n    "Tragedy": ["Tragedy"],\n    "Deception": ["Deception"],\n    "Hope": ["Hope"],\n    "Escap

In [267]:
output_formatted = output.content
# turn into json
output = json.loads(output_formatted)
len(output), output

(36,
 {'Providence': ['Providence'],
  'Power Dynamics': ['Power Dynamics', 'Power', 'Hierarchy'],
  'Emotions': ['Emotions', 'Emotional Themes'],
  'Suffering': ['Suffering', 'Misery'],
  'Social Themes': ['Social Themes', 'Societal Themes', 'Social Structure'],
  'Character Dynamics': ['Character Dynamics'],
  'Cultural Themes': ['Cultural Themes', 'Culture'],
  'Personal Development': ['Personal Development Themes', 'Adaptability'],
  'Identity': ['Identity', 'Personal Identity'],
  'Art and Perception': ['Art and Perception'],
  'Irony': ['Irony'],
  'Moral Themes': ['Moral Themes'],
  'Philosophical Themes': ['Philosophical Themes'],
  'Survival': ['Survival'],
  'Danger': ['Danger'],
  'Romantic Themes': ['Romantic Themes'],
  'Desire': ['Desire'],
  'Justice': ['Justice'],
  'Utopian Themes': ['Utopian Themes'],
  'Tragedy': ['Tragedy'],
  'Deception': ['Deception'],
  'Hope': ['Hope'],
  'Escape': ['Escape'],
  'Objectification': ['Objectification'],
  'Ambition': ['Ambition'],

In [268]:
# Capitalize output 
capitalized_output = {string.capwords(key): [string.capwords(name) for name in value] for key, value in output.items()}

# Process each scene
for c in characters:

    # Process group
    group = string.capwords(c.group)
    found_group = False

    # Check if the group is a key in the dictionary
    if group in capitalized_output:
        found_group = True
        c.group = group  # Capitalize the group

    # If not found in the dictionary, check the aliases
    if not found_group:
        matched_key = find_key_from_alias(group, capitalized_output)
        if matched_key != group:
            c.group = matched_key
            found_group = True
    if not found_group:
        c.group = group

### assign colors to characters

In [269]:
# now add colors for top characters
top_char_names = list(top_characters.keys())

all_res_colors = []
invalid_input = True
while invalid_input: # try until valid input
    # see if exception is thrown
    # filter all_json to only include characters in arr
    new_scenes = []
    for scene in scenes:
        include = False
        for character in scene["characters"]:
            if character["name"] in top_char_names:
                include = True
                break
        if include:
            new_scenes.append(scene)

    # take middle 126000 characters of new_scenes
    new_scenes_str = json.dumps(new_scenes)
    new_scenes_str = new_scenes_str[:126000]

    try:
        prompt = f"characters: {top_char_names}\nstory info: {new_scenes_str}\nyour output should contain exactly the same {len(top_char_names)} characters as in the original list. Make sure each character is assigned a unique color."
        res = colorlist_llm.invoke(prompt)
        all_res_colors.append(res)
        invalid_input = False
    except Exception as e:
        print("Exception thrown. Trying again.")
        print("Error:", e)
        invalid_input = True

In [270]:
# find duplicate colors
colorlist = []
for res in all_res_colors:
    colorlist += res.colorlist
    
unique_colors = []
duplicate_colors = []
for i, char in enumerate(colorlist):
    color = char.color
    if color not in unique_colors:
        unique_colors.append(color)
    else:
        print("duplicate color:", color, i)
        duplicate_colors.append(i)

In [271]:
colorinfo = {}

for c in colorlist:
    colorinfo[c.name] = {
        "color": c.color,
        "explanation": c.explanation
    }

colorinfo

{'Suffering': {'color': 'rgb(255, 0, 0)',
  'explanation': 'Red symbolizes the intense pain and suffering experienced by the characters.'},
 'Hope And Resilience': {'color': 'rgb(0, 255, 0)',
  'explanation': 'Green represents growth and the hope for a better future amidst adversity.'},
 'Love And Loyalty': {'color': 'rgb(255, 192, 203)',
  'explanation': 'Pink embodies love and loyalty, reflecting the deep emotional connections between characters.'},
 'Survival': {'color': 'rgb(255, 165, 0)',
  'explanation': 'Orange signifies the struggle for survival and the determination to overcome challenges.'},
 'Escape And Adventure': {'color': 'rgb(0, 0, 255)',
  'explanation': 'Blue represents the spirit of adventure and the desire to escape from dire situations.'},
 'Optimism': {'color': 'rgb(255, 255, 0)',
  'explanation': 'Yellow symbolizes optimism and the belief in a better tomorrow.'},
 'Disillusionment': {'color': 'rgb(128, 0, 128)',
  'explanation': 'Purple reflects the feelings of di

In [272]:
# remove duplicates
characters = [char for i, char in enumerate(characters) if i not in duplicate_indices]
print(len(characters))

199


In [273]:
character_info = []
for c in characters:

    # see if character is in color_info
    name = c.name
    color = ""
    explanation = ""
    if name in colorinfo:
        color = colorinfo[name]["color"]
        explanation = colorinfo[name]["explanation"]
    
    character_info.append({
        "name": name,
        "quote": c.quote,
        "group": c.group,
        "color": color,
        "explanation": explanation
    })

len(character_info), character_info

(199,
 [{'name': 'Optimism',
   'quote': '"It is demonstrable," said he, "that things cannot be otherwise than as they are; for all being created for an end, all is necessarily for the best end."',
   'group': 'Philosophical Themes',
   'color': 'rgb(255, 255, 0)',
   'explanation': 'Yellow symbolizes optimism and the belief in a better tomorrow.'},
  {'name': 'Social Status',
   'quote': 'The Baron was one of the most powerful lords in Westphalia, for his castle had not only a gate, but windows.',
   'group': 'Social Themes',
   'color': 'rgb(255, 215, 0)',
   'explanation': 'Gold represents social status and the hierarchies present in society.'},
  {'name': 'Innocence',
   'quote': 'Candide listened attentively and believed innocently; for he thought Miss Cunegonde extremely beautiful, though he never had the courage to tell her so.',
   'group': 'Emotions',
   'color': 'rgb(173, 216, 230)',
   'explanation': 'Light blue embodies innocence and the purity of the characters before thei

In [274]:
for name in need_to_add:
    character_info.append({
        "name": name,
        "quote": "",
        "group": "misc",
        "color": "",
        "explanation": ""
    })

len(character_info), character_info

(199,
 [{'name': 'Optimism',
   'quote': '"It is demonstrable," said he, "that things cannot be otherwise than as they are; for all being created for an end, all is necessarily for the best end."',
   'group': 'Philosophical Themes',
   'color': 'rgb(255, 255, 0)',
   'explanation': 'Yellow symbolizes optimism and the belief in a better tomorrow.'},
  {'name': 'Social Status',
   'quote': 'The Baron was one of the most powerful lords in Westphalia, for his castle had not only a gate, but windows.',
   'group': 'Social Themes',
   'color': 'rgb(255, 215, 0)',
   'explanation': 'Gold represents social status and the hierarchies present in society.'},
  {'name': 'Innocence',
   'quote': 'Candide listened attentively and believed innocently; for he thought Miss Cunegonde extremely beautiful, though he never had the courage to tell her so.',
   'group': 'Emotions',
   'color': 'rgb(173, 216, 230)',
   'explanation': 'Light blue embodies innocence and the purity of the characters before thei

## produce location summaries

In [275]:
# Pydantic
class LocationInfo(BaseModel):
    """Get information about a location in the story."""
    name: str = Field(description="Name of location.")
    quote: str = Field(description="Direct quote from the story that describes this location")
    emoji: str = Field(description="Emoji that represents this location (e.g., 🏰). Make sure each location has a different emoji.")

class LocationList(BaseModel):
    """Get overall information about the story."""
    locationList: list[LocationInfo] = Field(description="List of location details. One entry per location.")

locationList_llm = llm.with_structured_output(LocationList)

In [276]:
if not scene_details_exist:
    cropped_json = json.dumps(all_json)[:126000]
    location_prompt = f"locations: {all_locations_new}\nstory info: {cropped_json}\nyour output should contain exactly the same {len(all_locations_new)} locations as in the original list."
    invalid_input = True
    while invalid_input: # try until valid input
        # see if exception is thrown
        try:
            res = locationList_llm.invoke(location_prompt)
            invalid_input = False
        except Exception as e:
            print("Exception thrown. Trying again.")
            print("Error:", e)
            invalid_input = True

In [277]:
if not scene_details_exist:
    location_info = []
    for l in res.locationList:
        location_info.append({
            "name": l.name,
            "quote": l.quote,
            "emoji": l.emoji
        })

    print(len(location_info))
    print(location_info)

## generating the final json file!

In [278]:
if scene_details_exist:
    # load in locations from final_data
    with open(f"json/{og_story_name}/final_data.json") as f:
        location_info = json.load(f)["locations"]
    print("Loaded existing location info")

Loaded existing location info


In [279]:
# add character and location info to all_json
all_json["characters"] = character_info
all_json["locations"] = location_info
all_json

{'title': 'Candide',
 'type': 'Novel',
 'author': 'Voltaire',
 'year': '1759',
 'url': 'https://www.gutenberg.org/ebooks/19942',
 'image': 'https://m.media-amazon.com/images/I/61j3+Vjz0ZL._AC_UF1000,1000_QL80_.jpg',
 'num_chapters': 30,
 'num_scenes': 130,
 'num_characters': 199,
 'num_locations': 90,
 'chapters': [{'chapter': 'I HOW CANDIDE WAS BROUGHT UP IN A MAGNIFICENT CASTLE, AND HOW HE WAS EXPELLED THENCE.',
   'scenes': 3,
   'length': 5095,
   'num_lines': 69,
   'summary': "Candide's sheltered life in the Baron's castle is disrupted when he and Cunegonde share a forbidden kiss, while Pangloss's philosophical teachings unfold.",
   'importance': 0.8,
   'conflict': 0.43,
   'locations': {"Baron Of Thunder-ten-tronckh's Castle": 1,
    'Castle Park': 1,
    'Behind The Screen': 1},
   'characters': {'Optimism': 1,
    'Social Status': 1,
    'Innocence': 1,
    'Curiosity For Knowledge': 1,
    'Romantic Aspirations': 1,
    'Forbidden Love': 1,
    'Social Rejection': 1,
    'F

In [280]:
# save as json
with open(f"json/{story_name}/final_data.json", "w") as f:
    json.dump(all_json, f, indent=4)

# also save to ../src/data/ folder
data_file_name = og_story_name + "-new"
if analysis_type == "theme":
    data_file_name += "-themes"
with open(f"../src/data/{data_file_name}.json", "w") as f:
    json.dump(all_json, f, indent=4)

## compute time taken

In [281]:
end_time = time.time()
total_time = end_time - start_time

In [282]:
# convert to minutes
total_time_min = total_time / 60
print(f"Total time taken (minutes): {total_time_min:0.2f}")

Total time taken (minutes): 2.67
